### Find albums in Spotify and add songs to a playlist

In [1]:
import requests
import json
!pip install pandas
import pandas as pd
!pip install jsontable
import jsontable
import io
import time

### Set Global Date Variable
Allows specification of which source CSV to pull and helps to ensure unique playlists are created

In [2]:
pullDate = "20210220"

### Token Management
Right now I have to manually refresh the token via https://developer.spotify.com/console

In [19]:
TOKEN = "BQBhUWCm96W8llSlq7H0WCceQR-9MCKlseNScVs-Onn7XluF7ZZ2pQu3174B2upn6tKwonsRqr1yDJys1rYeE-0iQZaC3q51k8-_sASjFEZyg8AbYe8rNXzkbVbvY4zgOHXYArf-JkUtrDUB3wM638Yzsx06K0tehW159B9aQ9DaBp1-rKXz4MX4wb7ZYXChGQzTzIHI2GAY9AIrf8It7QgatPJ8HGTT"

### Header Construction

In [20]:
header = {"Authorization": "Bearer {}".format(TOKEN)}

### Request Construction
> Remember that Spotify will return a max of 20 objects by default, so artists with lots of albums might not truly be returning the most recent

In [5]:
artist = "bastille"
artist = artist.replace(" ", "%20")
artist = "artist:%22" + artist + "%22"
# album = "Bad Blood"
# album = album.replace(" ", "%20")
# album = "album:%22" + album + "%22"
# queryParam = album + artist
target = "https://api.spotify.com/v1/search?q={}&type=album".format(artist)
request = requests.get(target, headers = header).json()
print(json.dumps(request, indent=2))

{
  "albums": {
    "href": "https://api.spotify.com/v1/search?query=artist%3A%22bastille%22&type=album&offset=0&limit=20",
    "items": [
      {
        "album_type": "album",
        "artists": [
          {
            "external_urls": {
              "spotify": "https://open.spotify.com/artist/7EQ0qTo7fWT7DPxmxtSYEc"
            },
            "href": "https://api.spotify.com/v1/artists/7EQ0qTo7fWT7DPxmxtSYEc",
            "id": "7EQ0qTo7fWT7DPxmxtSYEc",
            "name": "Bastille",
            "type": "artist",
            "uri": "spotify:artist:7EQ0qTo7fWT7DPxmxtSYEc"
          }
        ],
        "available_markets": [
          "US"
        ],
        "external_urls": {
          "spotify": "https://open.spotify.com/album/64fQ94AVziavTPdnkCS6Nj"
        },
        "href": "https://api.spotify.com/v1/albums/64fQ94AVziavTPdnkCS6Nj",
        "id": "64fQ94AVziavTPdnkCS6Nj",
        "images": [
          {
            "height": 640,
            "url": "https://i.scdn.co/image/a

### Function Accepts CSV as Input and Pulls New Albums

Testing out artist searches that resulted in errors

In [11]:
newArtist = "L'FREAQ"
newArtist = newArtist.replace(" ", "%20")
newArtist = newArtist.replace("&", "%26")
newArtist = newArtist.replace("'", "%27")
newArtist = newArtist.replace(".", "%2E")
newArtist = "artist:%22" + newArtist + "%22"
target = "https://api.spotify.com/v1/search?q={}&type=album".format(newArtist)
print(target)
request = requests.get(target, headers = header).json()
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
print(len(df.index))
print(df)
# paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
# converter = jsontable.converter()
# converter.set_paths(paths)
# resultTable = converter.convert_json(request)
# df = pd.DataFrame(data=resultTable)
# df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
# df = df.drop(df.index[0])
# df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
# newest = pd.DataFrame(data=df.iloc[[0]])

https://api.spotify.com/v1/search?q=artist:%22L%27FREAQ%22&type=album
1
    0           1       2             3
0  ID  Album Name  Tracks  Release Date


In [14]:
url = "https://raw.githubusercontent.com/patrick-still/music-engine/main/New%20Music%20Source%20Files/iTunes%20Alternative%20{}.csv".format(pullDate)
# url = "https://raw.githubusercontent.com/patrick-still/music-engine/main/sampleArtists.csv"
fetch = requests.get(url).content
artists = pd.read_csv (io.StringIO(fetch.decode('utf-8')))
artists.style

newAlbums = pd.DataFrame()
errors = []

def pullAlbums(artistDF):
    
    global newAlbums
    global errors
    
    for index, row in artistDF.iterrows():
        # Wait five seconds to avoid 20 calls/minute limit on iTunes API
        # time.sleep(5)
        newArtist = row["Artist"]
        newArtist = newArtist.replace(" ", "%20")
        newArtist = newArtist.replace("&", "%26")
        newArtist = newArtist.replace("'", "%27")
        newArtist = newArtist.replace(".", "%2E")
        newArtist = "artist:%22" + newArtist + "%22"
        target = "https://api.spotify.com/v1/search?q={}&type=album".format(newArtist)
        request = requests.get(target, headers = header).json()
        paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
        converter = jsontable.converter()
        converter.set_paths(paths)
        resultTable = converter.convert_json(request)
        df = pd.DataFrame(data=resultTable)
        df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
        if len(df.index) > 1:
            df = df.drop(df.index[0])
            df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
            newest = pd.DataFrame(data=df.iloc[[0]])
            newAlbums = newAlbums.append(newest)
        else:
            errors.append(row["Artist"])
            continue

    print(newAlbums)
    # newAlbums.style
    # Would prefer to have the output styled in this manner but am getting "style is not supported for non-unique indices"
    print(errors)
    
pullAlbums(artists)

                  Album ID               Album Name Tracks Release Date
4   15APIHMOiJkPm11YDOaiuE    Acquainted with Night     10   2021-02-19
14  3l6zJfiaE7RjHgwK1In27b       American Standards     12   2021-02-19
5   0kxIBVwMO6suZGKMOEriVG    As The Love Continues     11   2021-02-19
5   6x5HKPptybISjArVIpo5bN          At Your Funeral      1   2021-02-19
4   2V3TNPXvdNjqXAHPKYXUAT              Begin Again      1   2021-02-19
..                     ...                      ...    ...          ...
4   0NMpswZbEcswI3OIe6ml3Y             Dirty (Live)      2   2021-02-16
4   26suVOYpQ5SYiqo7CHyE46                eye 2 eye      1   2021-02-16
4   1vKrWX2mgtF6ryafuZQ5jl           Girl Next Door      1   2021-02-16
3   3y827hRkzzvvsUHmBxagYp  A Soft Inland Murmuring      1   2021-02-16
1   2S5Eg0cpDMgHpyrrEcL4vR        love isn't enough      6   2021-02-15

[96 rows x 4 columns]
["L'FREAQ", 'Lady HD & Eraserhood Sound', 'Ouri & Mobilegirl', 'Porridge Radio & piglet']


### Pull Album ID Out of Response
Parse some key fields, and sort by both release date and number of tracks

In [5]:
paths = [{"$.albums.items.id":"ID"}, {"$.albums.items.name":"Album Name"}, {"$.albums.items.total_tracks":"Tracks"}, {"$.albums.items.release_date":"Release Date"}]
converter = jsontable.converter()
converter.set_paths(paths)
resultTable = converter.convert_json(request)
df = pd.DataFrame(data=resultTable)
df.columns = ["Album ID", "Album Name", "Tracks", "Release Date"]
df = df.drop(df.index[0])
df = df.sort_values(["Release Date", "Tracks"], ascending = (False, False))
newest = pd.DataFrame(data=df.iloc[[0]])
newAlbums = pd.DataFrame()
newAlbums = newAlbums.append(newest)
newAlbums.style

,Album ID,Album Name,Tracks,Release Date
2,5G6oMu9zNW2acdV0lqzI3L,All This Bad Blood,25,2013-01-01


### Create Spotify Playlist
> Be careful here because Spotify will let you create playlists with the same name and description, assigning it a new playlist ID

In [21]:
userID = "t1e947vjw65dme71utp690f50"
createPlaylistTarget = "https://api.spotify.com/v1/users/{}/playlists".format(userID)
payload = {"name": pullDate, "description": "Created from Jupyter notebook", "public": False}
payload = json.dumps(payload, indent=2)
createPlaylistPOST = requests.post(createPlaylistTarget, headers = header, data = payload).json()
print(json.dumps(createPlaylistPOST, indent=2))

{
  "collaborative": false,
  "description": "Created from Jupyter notebook",
  "external_urls": {
    "spotify": "https://open.spotify.com/playlist/2gYyxNXZL1heqZGx8CDUDN"
  },
  "followers": {
    "href": null,
    "total": 0
  },
  "href": "https://api.spotify.com/v1/playlists/2gYyxNXZL1heqZGx8CDUDN",
  "id": "2gYyxNXZL1heqZGx8CDUDN",
  "images": [],
  "name": "20210220",
  "owner": {
    "display_name": "Patrick",
    "external_urls": {
      "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
    },
    "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
    "id": "t1e947vjw65dme71utp690f50",
    "type": "user",
    "uri": "spotify:user:t1e947vjw65dme71utp690f50"
  },
  "primary_color": null,
  "public": false,
  "snapshot_id": "MSwwMzk2MGZlZTYyYWQzYTcxZWYwMmE0ZWRjNjA3NzU1YzIxMDc2OWQy",
  "tracks": {
    "href": "https://api.spotify.com/v1/playlists/2gYyxNXZL1heqZGx8CDUDN/tracks",
    "items": [],
    "limit": 100,
    "next": null,
    "offset":

Retrieve User Information

In [19]:
getUserTarget = "https://api.spotify.com/v1/users/{}".format(userID)
getUser = requests.get(getUserTarget, headers = header).json()
print(json.dumps(getUser, indent=2))

{
  "display_name": "Patrick",
  "external_urls": {
    "spotify": "https://open.spotify.com/user/t1e947vjw65dme71utp690f50"
  },
  "followers": {
    "href": null,
    "total": 1
  },
  "href": "https://api.spotify.com/v1/users/t1e947vjw65dme71utp690f50",
  "id": "t1e947vjw65dme71utp690f50",
  "images": [
    {
      "height": null,
      "url": "https://i.scdn.co/image/ab6775700000ee85804e93a20b51ae91923b1a51",
      "width": null
    }
  ],
  "type": "user",
  "uri": "spotify:user:t1e947vjw65dme71utp690f50"
}


### Fetch Track IDs for Album
Store tracks in a dataframe and potentially sort on popularity (not seeing clear metadata to support this) 
Probably makes sense to add to the playlist in this loop so that it's adding chunks of tracks per album and any kind of error handling can be associated with the album.

In [22]:
playlistID = "2gYyxNXZL1heqZGx8CDUDN"
newTracks = pd.DataFrame()

def assembleTracks(albumDF, playlist):
    
    global newTracks
    
    # Assemble request to add the album's tracks to the target playlist which is passed as function parameter
    addTracksTarget = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlist)
    
    for index, row in albumDF.iterrows():
        # Wait five seconds to avoid rate limit
        # time.sleep(5)
        newAlbumID = row["Album ID"]
        fetchTrackTarget = "https://api.spotify.com/v1/albums/{}/tracks".format(newAlbumID)
        fetchTrack = requests.get(fetchTrackTarget, headers = header).json()
        # parse JSON for appropriate track metadata to store
        paths = [{"$.items.id":"Track ID"}, {"$.items.name":"Track Name"}]
        converter = jsontable.converter()
        converter.set_paths(paths)
        trackTable = converter.convert_json(fetchTrack)
        df = pd.DataFrame(data=trackTable)
        df.columns = ["Track ID", "Track Name"]
        df = df.drop(df.index[0])
        newTracks = newTracks.append(df)

        # Assemble track URIs to add to payload for request to add tracks to playlist
        trackURIs = []
        
        for index, row in df.iterrows():
            selectedTrack = row["Track ID"]
            selectedTrackURI = "spotify:track:{}".format(selectedTrack)
            trackURIs.append(selectedTrackURI)
        
        # Assemble payload
        payload = {"uris": trackURIs, "position":"null"}
        payload = json.dumps(payload, indent=2)
        
        # Execute request
        addTracks = requests.post(addTracksTarget, headers = header, data = payload).json()
        
    print(newTracks)
        
assembleTracks(newAlbums, playlistID)

                  Track ID                      Track Name
1   1yF9soeWa8OIjOATKYfazt                       Blue Vein
2   56hyqliBwT15WP1sCdOp4E  Every Star Shivers in the Dark
3   5txG3nEaDQCRhvALDeGu3f           Acquainted with Night
4   5Q2htxnOQCerYTnk3bA4Dv                     White Wings
5   5007rI4hVHvetp68Ln9nx9      How Far Is It to the Grave
..                     ...                             ...
2   4cqR0Q6L9hCLfp5XMt2rji                            time
3   58i55RrO1M44vel8OxWBDM                      work to do
4   7pfV3LWwlVKM8np5sZA6qQ                            lady
5   48tTn8RR9CYC3K3c92UrSR                       very busy
6   1TcmV7DPMnkSrTPRmOipRN                     faded naked

[399 rows x 2 columns]


In [25]:
pd.set_option("display.max_rows", None)
display(newTracks)

,Track ID,Track Name
1,1yF9soeWa8OIjOATKYfazt,Blue Vein
2,56hyqliBwT15WP1sCdOp4E,Every Star Shivers in the Dark
3,5txG3nEaDQCRhvALDeGu3f,Acquainted with Night
4,5Q2htxnOQCerYTnk3bA4Dv,White Wings
5,5007rI4hVHvetp68Ln9nx9,How Far Is It to the Grave
6,2kTEfe945y7wLHse1XH9dL,For No One For Now
7,7fGzcQrX9V3snwl5QVndxf,Sliding Doors & Warm Summer Roses
8,7HpTCXpWi5HosPuGDIS589,Third Floor Window
9,5736AfyLBfY9TGdIvcG2Fb,Let Me Live by the Side of the Road
10,0jyFHQ1VITvwx2RiR4oKpH,Some Sunny Day


### Add Tracks to a Playlist
POST https://api.spotify.com/v1/playlists/{playlist_id}/tracks  
Need the playlist ID and a list of track URIs  

> Folded this into the assembleTracks function, so the logic below is now just a reference for making a single call

In [9]:
playlistID = "25NbFuQM1kDleGz1bjGJ2x"
addTrackTarget = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlistID)
payload = {"uris": ["spotify:track:57As4lhkpp6tB422daefKK","spotify:track:3ePeWP6f8ccQtF7wayoNrD"], "position":"null"}
payload = json.dumps(payload, indent=2)
addTrack = requests.post(addTrackTarget, headers = header, data = payload).json()
print(json.dumps(addTrack, indent=2))

{
  "snapshot_id": "MTIsMzBmODE0NzUwOGVlNGJmZWZmZWYyMjExOGI2NzYyNjQzNzY5ZDExNA=="
}
